In [ ]:
import random
import requests
import datetime
import pandas as pd
import dapla
import pyspark
random.seed(20)

In [ ]:
# henter klassifikasjoner fra KLASS
pd.set_option('display.max_columns', None)
# pd.options.display.max_colwidth = 150

klasstabell = 131  # Kommuner
klassdato = datetime.datetime.now().strftime("%Y-%m-%d")
response_json1 = requests.get(f'https://data.ssb.no/api/klass/v1/classifications/{klasstabell}/codesAt.json?date={klassdato}').json()
pd_kommnr = pd.DataFrame(response_json1['codes'])
klasstabell = 91  # landkoder
response_json2 = requests.get(f'https://data.ssb.no/api/klass/v1/classifications/{klasstabell}/codesAt.json?date={klassdato}').json()
pd_landkoder = pd.DataFrame(response_json2['codes'])

# oversikt over kommunestørrelser 1986 -> 
data = pd.read_csv("https://data.ssb.no/api/v0/dataset/26975.csv?lang=no", sep=';', decimal=',', encoding = "ISO-8859-1", )
x = data['år'] == 2019
print(x)
data2 = data[data['år'] == 2019]
print(type(data2))

data2['region1'] = data2['region'].str.slice(2, 6)
print(data2['region1'])
data2['vekt'] = data2['07459: Befolkning, etter region, år og statistikkvariabel'].div(data2['07459: Befolkning, etter region, år og statistikkvariabel'].sum())
print(data2['vekt'])
data2

In [ ]:
# Testsett befolkningsdata med vekting
r = random.sample(range(10000000000,99999999999), 100)
# random.sample selects an item from the range. sample assures that there will be no repeated items.
# "price": random.randrange(1,10000) / 100 returner integer som kan benyttes i en operasjon
fnr = [str(x) for x in r]
land = pd_landkoder['code']
kommnr = pd_kommnr['code']
testdata = [
    {
        "pseudonym": random.choice(fnr),
        "foedland": random.choices(['000',random.choice(land)],weights=[85,15], k=1)[0], #Vektet -> 85% velges '000' (norske) for de resterende 15% velges tilfeldig kode fra land 
        "kjoenn": random.randrange(1,3),
        "kommnr": random.choices(data2['region1'].values.tolist(),weights=data2['vekt'].values.tolist(),k=1)[0],
        "mor_fnr": random.choice(fnr)
    } 
    for _ in range(100)
]

In [ ]:
testdata_pd = pd.DataFrame(testdata)

In [ ]:
print(testdata_pd)

In [ ]:
# typisk enkelt forløpsdatasett i microdata.no. Viser mottatt ytelse for eks.
r = random.sample(range(10000000000,99999999999), 100)
dager = random.sample(range(1000), 100)
fnr = [str(x) for x in r]
datoliste = pd.date_range(start="1992-01-01", end="2020-12-31" ,freq="D")
test = [
    {
        "pseudonym": random.choice(fnr),
        "kode": 1,
        "start": random.choices(datoliste)[0],
        "dager": random.choices(dager)[0]
    } 
    for _ in range(100) 
]

# bruker funksjonalitet i pandas for å lage sluttdatoene:
testdata_forlop_pd = pd.DataFrame.from_dict(test)
testdata_forlop_pd["stop"] = pd.to_datetime(testdata_forlop_pd["start"]) + pd.to_timedelta(testdata_forlop_pd['dager'],'d')
del testdata_forlop_pd['dager']

In [ ]:
pd.set_option('display.max_rows', 100)
testdata_forlop_pd

In [ ]:
# ser på count, har ikke null verdier så får ikke frem forskjellen
print(f'Ant. unike:',testdata_forlop_pd['pseudonym'].nunique())
print(f'Ant. (ikke tell "null"):',testdata_forlop_pd['pseudonym'].count())
print(f'Ant. totalt:',testdata_forlop_pd['pseudonym'].size)

In [ ]:
# samlet
testdata_forlop_pd.loc[testdata_forlop_pd['kode']==1,'pseudonym'].agg(['nunique','count','size'])

In [ ]:
# kolonne begrensning
testdata_forlop_pd.query('kode == 1')['pseudonym'].agg(['nunique','count','size'])

In [ ]:
testdata_forlop_pd.nunique()

In [ ]:
pd.set_option('display.max_rows', None)
testdata_forlop_pd.groupby('pseudonym').agg(['count', 'size', 'nunique']).stack()

In [ ]:
# litt dårlig måte, men kan se duplikater på pseudonym
testdata_forlop_pd.value_counts()